In [2]:
import numpy as np
import pandas as pd
import os

from pathlib import Path
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

In [3]:
class main_class_data():
    def __init__(self,file_path):
        self.file_path=file_path
        
    def data_set(self):
        image_link=list(self.file_path.glob(r'**/*.tif'))
        image_labels=[x.parents[0].stem for x in image_link]
        
        df=pd.DataFrame(list(zip(image_link,image_labels)),columns=['ImageLinks','ImageLabels'])
        df=df.sample(frac=1,random_state=1).reset_index(drop=True)
        __,axes=plt.subplots(5,5,figsize=(10,10))
        for i,ax in enumerate(axes.flat):
            ax.imshow(cv2.imread(str(df.ImageLinks[i])))
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(df.ImageLabels[i])
        plt.show()
        
        return df

In [4]:
img_path=Path('/kaggle/input','weed-detection-in-soybean-crops')
df=main_class_data(img_path)
df=df.data_set()

In [5]:
#create Data Frame without soyben

df_weed=df[df.ImageLabels!='soybean']
df_weed.shape

In [6]:
df_weed['ImageId']=preprocessing.LabelEncoder().fit_transform(df_weed.ImageLabels)

In [7]:
df_weed.ImageId.value_counts()

In [8]:
x=df_weed.ImageLinks
y=df_weed.ImageId
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.01,random_state=101)

In [9]:
X_train=[]
for i in xtrain:
    img_pre=cv2.imread(str(i))
    img_pre=cv2.resize(img_pre, (150, 150))
    X_train.append(img_pre)
    
X_train=np.array(X_train)
ytrain=to_categorical(ytrain)

In [10]:
# extra 
X_test=[]
for i in xtest:
    img_pre1=cv2.imread(str(i))
    img_pre1=cv2.resize(img_pre1, (150, 150))
    X_test.append(img_pre1)
    
X_test=np.array(X_test)
ytest=to_categorical(ytest)

In [11]:
efficient = EfficientNetB0(weights='imagenet',include_top = False, input_shape=(150, 150, 3)) 

In [12]:
model = efficient.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)

model = tf.keras.layers.Dense(3, activation = "softmax")(model)

model = tf.keras.models.Model(inputs = efficient.input , outputs = model)

In [13]:
model.compile(loss = 'categorical_crossentropy', 
             optimizer = "Adam", 
              metrics=["accuracy"]
             )

In [14]:
tensorboard = TensorBoard(log_dir="logs")
checkpoint = ModelCheckpoint("efficient.h5", 
                            monitor='val_accuracy', 
                            mode = "auto", 
                            verbose = 1,
                            save_best_only =True)
reduce_LR = ReduceLROnPlateau(monitor="val_accuracy", 
                             factor = 0.3, 
                             patience = 2, 
                             min_delta=0.001, 
                             mode = "auto", 
                            verbose = 1)

In [110]:
#fit model only weed

In [15]:
history = model.fit(X_train, ytrain,
                  validation_split = 0.1, 
                  epochs = 10, 
                  verbose =1, 
                  batch_size = 32, 
                  callbacks = [tensorboard, checkpoint, reduce_LR])

In [16]:
# create data frame only soybean

df_soybean=df[df.ImageLabels=='soybean']
df_soybean.shape

In [18]:
test_image=[]
for i in df_soybean.ImageLinks:
    img_pre=cv2.imread(str(i))
    img_pre=cv2.resize(img_pre, (150, 150))
    test_image.append(img_pre)
    
test_image=np.array(test_image)

In [111]:
# predict with target values

In [21]:
prdre = model.predict(test_image)

In [70]:
# plot for weed and weed free

plt.figure(figsize=(18, 18))   
for i in range(25):
    prd = np.argmax(prdre, axis = 1)[i]
    #titel_lbl=name[prd]
    titel_lbl=('soyabean' if prd==0 else 'WeedDetect')
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(test_image[i])
    plt.title(titel_lbl)
    plt.axis("off")

In [107]:
# create report data frame 

list_name=[]
for i in range(14752):
    prd1 = np.argmax(prdre, axis = 1)[i]
    titel_lbl1='Weed Free' if prd1==0 else 'Weed Detect'
    #print(titel_lbl1.split())
    list_name.append(titel_lbl1)
    
df_soybean.WeedDetection=list_name
df_soybean.head(10)

In [109]:
# plot weed and free soybean
df_soybean.groupby(['ImageLabels','WeedDetection']).agg({'WeedDetection': 'count'}).plot(kind='bar',figsize = (12,8), grid = True, color = 'teal')

In [130]:
print('Total Saybean image :', df_soybean.ImageLabels.value_counts()[0])
print('Total Weed Detect image :', df_soybean.WeedDetection.value_counts()[0])
print('Total Weed Free image :', df_soybean.WeedDetection.value_counts()[1])